In [28]:
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
import pandas as pd
load_dotenv()

True

In [32]:
# grab the zip code we've saved as an environment variable
zip_code = os.environ.get('ZIP')

driver = webdriver.Firefox()
driver.maximize_window()
# access Whole Foods' products that are on sale via selenium
driver.get("https://www.wholefoodsmarket.com/products/all-products?featured=on-sale")
assert "Whole Foods" in driver.title

# wait for the page to load
wait = WebDriverWait(driver, 3)

# try/except in case selenium opens us logged in for some reason
try:
    # we create a variable to grab the search field
    store = driver.find_element(by=By.ID, value="pie-store-finder-modal-search-field")
    # ensure it's empty
    store.clear()
    # selenium adds our zip code into the text field
    store.send_keys(zip_code)
    store.send_keys(Keys.RETURN)
    
    # wait until a nearby Whole Foods shows up as a clickable option
    closest = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'li.wfm-search-bar--list_item:nth-child(1)')))
    closest.click()
except:
    NoSuchElementException('Either you already have the lcoation or there\'s an error')

# the on sale products page does not show all of the products
# in fact, there are so many deals on sale that I have no idea
# what the exact amount is, so I limit the amount of extra
# product pages that we load more to 5

# we wait for the load more button to load since it doesn't initially
load_more = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.w-button:nth-child(5)")))
for _ in range(5):
    # we click 5 times
    load_more.click()

In [40]:
# grabbing the html source after loading more deals
# than we would have gotten after initially loading
# the page
content = driver.page_source

# make working with the html easier with BeautifulSoup
dom = BeautifulSoup(content, 'html.parser')
print(dom.body)

<bound method Tag.prettify of <body><noscript><iframe height="0" sandbox="" src="https://www.googletagmanager.com/ns.html?id=GTM-MMVR7TC" style="display:none;visibility:hidden" width="0"></iframe></noscript><div id="__next"><main><header class="wfm-global-header"><a class="w-skip-link" data-testid="skip-link" href="#main-content"><span>Skip main navigation</span></a><nav aria-label="main navigation" class="wfm-global-nav" id="header"><div class="wfm-global-nav__group"><div class="wfm-hamburger-wrapper w-hidden-in-desktop"><button aria-expanded="false" aria-haspopup="true" class="wfm-hamburger" data-testid="hamburger-menu"><span aria-hidden="true" class="wfm-hamburger__box"><span class="wfm-hamburger__inner"></span></span><span class="sr-only">Toggle Site Navigation Menu</span></button></div><div class="wfm-global-nav__logo"><a href="/"><span class="w-visually-hidden">Home</span></a></div></div><div class="wfm-global-nav__divider"></div><div class="w-global-menu w-hidden-in-tablet w-hid